In [7]:
# train - 3p trb blk 열 데이터 -> pos를 예측하는 knn 모델 생성
# test - 3p trb blk 열 데이터를 knn 모델에 입력 -> 예측 결과와 정답 비교 -> 정확도 출력

import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


# 파일 로드

with open('basketball_test.pkl', 'rb') as test_data:
    test = pd.read_pickle(test_data)

with open('basketball_train.pkl', 'rb') as train_data:
    train = pd.read_pickle(train_data)

train_test_data = [train, test]


# 전처리

pos_mapping = {'SG': 0, 'C': 1}

for dataset in train_test_data:
    dataset['Pos'] = dataset['Pos'].map(pos_mapping)
    dataset.drop('Player', axis=1, inplace=True)

col = ['3P', 'TRB', 'BLK']

train_data = train[col]
test_data = test[col]
target = train['Pos']


# 모델 생성

clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(train_data, target)


# 예측

prediction = clf.predict(test_data)


# 정확도

k_fold = KFold(n_splits=10, shuffle=True, random_state=0)
scoring = 'accuracy'
score = cross_val_score(clf, train_data, target, cv=k_fold, n_jobs=1, scoring=scoring)
round(np.mean(score) * 100, 2)


# 답안지 작성

submission = pd.DataFrame({
        "Pos" : test.Pos,
        "predict_pos": prediction
    })

submission.to_csv('pos_submission.csv', index=False)